# Setup
Hello.  Connecting to Oracle in Python is a trip and a half.  Connecting to SaaS or to Bb Data is supposedly easier - and I'm working to ensure that importing things works easily for everyone.

In [10]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd
from pprint import pprint

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

### Simple example
This selects 5 users and imports into a pandas dataframe

In [4]:
query = """
SELECT user_id
FROM BB_BB60.USERS
FETCH FIRST 5 ROWS ONLY
"""

users = pd.read_sql_query(query, con=engine)

In [5]:
users.head()

,user_id
0,04292595_testuser
1,04294676_student
2,04578241_student
3,88smercer
4,_01018694_isaak


### Snowflake setup
Connecting to snowflake is much easier.  Everything in this presentation is focused on Oracle PL/SQL, but if you are on SaaS - many examples can be translated and run against snowflake or postgres.  Here's how to connect via Snowflake.

In [6]:
from snowflake.sqlalchemy import URL
snow_engine = create_engine(URL(
    account = ge('snow_account'),
    user = ge('snow_user'),
    password = ge('snow_pass'),
    database = ge('snow_db'),
    warehouse = ge('snow_wh')
))


query = """
select
    year(lt.start_date) as term_year,
    ifnull(lt.name, 'Unknown Term') as term,
    replace(ifnull(h2.name, 'No Parent'),'NoName', 'Institution') as hierarchy_parent_node,
    ifnull(h1.name, 'No Node') as hierarchy_node,
    count(distinct lc.id) as course_count
from cdm_lms.course lc
inner join cdm_lms.institution_hierarchy_course ihc
    on lc.id = ihc.course_id
    and ihc.primary_ind = 1
    and ihc.row_deleted_time is null
left join cdm_lms.institution_hierarchy h1
    on ihc.institution_hierarchy_id = h1.id
left join cdm_lms.institution_hierarchy h2
    on h1.institution_hierarchy_parent_id = h2.id 
left join cdm_lms.term lt
    on lt.id = lc.term_id
group by
    year(lt.start_date),
    h1.name,
    h2.name,
    lt.name
order by
    year(lt.start_date),
    lt.name,
    h1.name
"""
example = pd.read_sql(query, con=snow_engine)                    

In [7]:
example.head()

,term_year,term,hierarchy_parent_node,hierarchy_node,course_count
0,2014.0,Fall 2014,Institution,Beta,10
1,2014.0,Fall 2014,Institution,Chemical Engineering,15
2,2014.0,Fall 2014,Institution,English,27
3,2014.0,Fall 2014,Institution,First Year,26
4,2014.0,Fall 2014,Institution,Honors,17


# From files example

In [8]:
query = ''
with open('./queryfiles/no_access.sql','r') as f:
    query = f.read()

In [11]:
pprint(query)

("SELECT SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT,\n"
 "       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE,\n"
 "       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,\n"
 '       cm.COURSE_ID,\n'
 '       cm.AVAILABLE_IND,\n'
 '       MAX(cu.LAST_ACCESS_DATE) last_access,\n'
 '       SUM(students.students)/COUNT(students.students) student_count,\n'
 "       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,\n"
 "       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by "
 'u.email) names\n'
 '\n'
 'FROM BB_BB60.USERS u inner join BB_BB60.COURSE_USERS cu on u.pk1 = '
 'cu.USERS_PK1\n'
 'inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.pk1\n'
 'left join (select crsmain_pk1, count(pk1) students\n'
 '    from bb_bb60.COURSE_USERS\n'
 "    where role='S'\n"
 '    and row_status = 0\n'
 '    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1\n'
 '\n'
 "WHERE cm.COURSE_NAME LIKE '2020' || :season ||

# Creating Pandas DataFrames from SQL
Pandas dataframes are the standard for working with data in python.  They can read in a variety of data files.  In this case, we are reading in SQL and querying the database.  Most of these queries seek to get a LOT of information, that can be paired down after the fact.  The same can be acheived directly by editing the query - but pandas helps us combine information from other sources, and write to files to a variety of other formats.

In [14]:
access_df = pd.read_sql(query, 
                        params={'season':'Fall'}, 
                        con=engine, 
                        parse_dates=['last_access'])

In [15]:
access_df.head()

,course_unit,course_type,course_name,course_id,available_ind,last_access,student_count,emails,names
0,SOC,LEC,2020Fall-SOC 104 Elements of Sociology LEC,4209-12904,N,NaT,659,lmw@ku.edu,Lisa-Marie Wright
1,LDST,LEC,2020Fall-LDST 301 It's On Us: Gender Based Vi...,4209-28400,N,2020-07-06 13:39:24,535,soheath@ku.edu,Sony Heath
2,BIOL,LEC,2020Fall-BIOL 150 Principles of Molecular and ...,4209-10381,N,2020-07-01 11:19:06,525,ehotze@ku.edu;trivers@ku.edu,Eileen Hotze;Trevor Rivers
3,BIOL,LEC,2020Fall-BIOL 100 Principles of Biology LEC,4209-10339,N,NaT,522,kichler@ku.edu;tmarria@ku.edu,Kristina Holder;Tara Marriage
4,BUS,LEC,2020Fall-BUS 120 Emerging Topics in Business LEC,4209-29443,N,NaT,504,jfitchet@ku.edu;kkammerer@ku.edu,Janalee Fitchett;Karl Kammerer


In [16]:
no_access_df = access_df[(access_df['course_type']=='LEC') & 
                         (access_df['available_ind']=='Y') & 
                         (access_df['last_access'] < pd.to_datetime("2020-07-06"))]

In [17]:
no_access_df.head()

,course_unit,course_type,course_name,course_id,available_ind,last_access,student_count,emails,names
15,AE,LEC,2020Fall-AE 290 Aerospace Colloquium LEC,4209-10055,Y,2020-03-28 19:26:27,287,rtaghavi@ku.edu,Ray Taghavi
157,EECS,LEC,2020Fall-EECS 645 Computer Architecture LEC,4209-15749,Y,2020-06-08 14:33:24,83,esam@ku.edu,Esam Eldin Aly
255,AE,LEC,2020Fall-AE 571 Fundamentals of Airplane Recip...,4209-10069,Y,2020-03-28 19:30:27,55,rohith.giridhar91@ku.edu;rtaghavi@ku.edu,Rohith Giridhar;Ray Taghavi
452,PSYC,LEC,2020Fall-PSYC 375 Cognitive Neuroscience LEC,4209-22108,Y,2020-07-04 15:27:28,37,mattrosenthal@ku.edu,Matthew Rosenthal
524,PHSX,LEC,2020Fall-PHSX 531 Electricity and Magnetism LEC,4209-12505,Y,2020-04-28 19:44:39,34,medvedev@ku.edu,Mikhail Medvedev


# Creating Excel Files from Pandas

In [18]:
from datetime import datetime
date = datetime.today().strftime("%m-%d")
no_access_df.to_excel(f'{date}-NoAccess.xlsx')

# Putting those excel files into onedrive

### Setup MS Connection

In [19]:
client_id = ge('client_id')
authority_id = ge('authority_id')
client_credential = ge('client_credential')
drive_id = ge('drive_id')
item_id = ge('item_id')

In [20]:
from msal import ConfidentialClientApplication
import requests

app = ConfidentialClientApplication(client_id=client_id,
                                    authority=f"https://login.microsoftonline.com/{authority_id}",
                                    client_credential=client_credential)

result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
token = result['access_token']
s = requests.Session()
http_headers = {'Authorization': 'Bearer ' + result['access_token'],
                'Accept': 'application/json',
                'Content-Type': 'application/json'}
s.headers.update(http_headers)
base_url = 'https://graph.microsoft.com/beta'

In [21]:
r = s.put(f"{base_url}/drives/{drive_id}/items/{item_id}:/{date}-NoAccess.xlsx:/content", 
            data=open(f'{date}-NoAccess.xlsx','rb'))

In [22]:
r.json()

{'@odata.context': "https://graph.microsoft.com/beta/$metadata#drives('b%212gKIaGIsfU-pWCgyfCEltNbxnxaD5cVLmJ4HlMYYp737N6aHJRFRRoUKxHMP1jfc')/items/$entity", '@microsoft.graph.downloadUrl': 'https://kansas-my.sharepoint.com/personal/m500d520_home_ku_edu/_layouts/15/download.aspx?UniqueId=16c15b56-4ed4-41c3-ae3b-8fc0b6f827bb&Translate=false&tempauth=eyJ0eXAiOiJKV1QiLCJhbGciOiJub25lIn0.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTBmZjEtY2UwMC0wMDAwMDAwMDAwMDAva2Fuc2FzLW15LnNoYXJlcG9pbnQuY29tQDNjMTc2NTM2LWFmZTYtNDNmNS1iOTY2LTM2ZmVhYmJlM2MxYSIsImlzcyI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMCIsIm5iZiI6IjE1OTQ4MzgyMTMiLCJleHAiOiIxNTk0ODQxODEzIiwiZW5kcG9pbnR1cmwiOiJrKzM1bVdmZDRveXVIc0NHWVZ3L0h6UnR3NFFlQkJMVzJ1QXRJU1hBMUNvPSIsImVuZHBvaW50dXJsTGVuZ3RoIjoiMTUwIiwiaXNsb29wYmFjayI6IlRydWUiLCJjaWQiOiJOakZrWTJFM05tUXRPVEpoWVMwMFlqUTFMVGc1WVdVdE9UUTBaalprTkRrMFlqRTIiLCJ2ZXIiOiJoYXNoZWRwcm9vZnRva2VuIiwic2l0ZWlkIjoiTmpnNE9EQXlaR0V0TW1NMk1pMDBaamRrTFdFNU5UZ3RNamd6TWpkak1qRXlOV0kwIiwiYXBwX2Rpc3BsYXluYW1lIjoiS